In [1]:
import os
import json
import arcgis
import numpy as np
import datetime
import time
import smtplib, ssl
from email.message import EmailMessage
from arcgis.geometry import Point
from arcgis.geometry import Geometry
from arcgis.gis import GIS

### Acceder a la organización y los elementos en la capa

In [2]:
gis = GIS("URL/portal", "USER", "PASSWORD", verify_cert=False)
reubicacion_item = gis.content.get("d2f19956d2e948bebaa67919cba50e54")
reubicacion_layer = reubicacion_item.layers

### Función de envío mail

In [ ]:
def send_mail(elemento, fecha, punto, tipo, asunto):
    print("Preparando mail...")
    port = 465 
    sender_email = "esvalmarkupnotification@gmail.com" #correo origen
    receiver_email = ["jcordovafredes@outlook.com"] #correo destinatario
    smtp_server = "smtp.gmail.com"
    message = """
    Estimad@,
    <br>
    <br>
    Con fecha {date}; se ha Aceptado la {operation_type} de un elemento tipo {element}.
    <br>
    <br>
    Puede dirigirse al mapa haciendo click en el link: <a href= URL del MAPA>Mapa</a>
    <br>
    <br>
    Este correo es generado automáticamente, no responder."""
    
    msg = EmailMessage()
    msg.set_content(message.format(element=elemento, date=fecha, point=punto, operation_type=tipo), "html")
    msg['Subject'] = '{subject} de elemento aceptada'.format(subject=asunto)
    msg['From'] = sender_email
    msg['To'] = receiver_email
    password = "tvt.2021"
    context = ssl.create_default_context()

    with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
        server.login(sender_email, password)   
        server.send_message(msg)
        server.quit()

### Startup

In [5]:
element_layer_old = reubicacion_layer[0].query(as_df=True)
element_layer_new = reubicacion_layer[0].query(as_df=True)

### Loop

In [ ]:
while 1:
    for index, row in element_layer_old.iterrows():
        index_new_array = np.where(element_layer_new["ID_REGISTRO"]==row["ID_REGISTRO"])[0][0]
        if element_layer_new.iloc[index_new_array].ESTADO_REUBICACION != row["ESTADO_REUBICACION"] and element_layer_new.iloc[index_new_array].ESTADO_REUBICACION == "Aceptado" and element_layer_new.iloc[index_new_array].TIPO_TEMATIZACION == "FIN":
           # print("Se detectó un cambio en para el índice: " + str(index_new_array))
           # print("El elemento tiene los siguientes datos: ")
           # print(element_layer_new.iloc[index_new_array])
            element = element_layer_new.iloc[index_new_array].TIPO_ELEMENTO
            fecha = str(datetime.datetime.now())[0:16]
            point= element_layer_new.iloc[index_new_array].SHAPE
            punto = str(point.x)+","+str(point.y)+","+str(point.spatialReference["wkid"])
            send_mail(element,fecha,punto, "reubicación", "Reubicación")
        elif element_layer_new.iloc[index_new_array].ESTADO_REUBICACION != row["ESTADO_REUBICACION"] and element_layer_new.iloc[index_new_array].ESTADO_REUBICACION == "Aceptado" and element_layer_new.iloc[index_new_array].TIPO_TEMATIZACION == "CREAR":
           # print("Se detectó un cambio en para el índice: " + str(index_new_array))
           # print("El elemento tiene los siguientes datos: ")
           # print(element_layer_new.iloc[index_new_array])
            element = element_layer_new.iloc[index_new_array].TIPO_ELEMENTO
            fecha = str(datetime.datetime.now())[0:16]
            point= element_layer_new.iloc[index_new_array].SHAPE
            punto = str(point.x)+","+str(point.y)+","+str(point.spatialReference["wkid"])
            send_mail(element,fecha,punto, "creación", "Creación")
        elif element_layer_new.iloc[index_new_array].ESTADO_REUBICACION != row["ESTADO_REUBICACION"] and element_layer_new.iloc[index_new_array].ESTADO_REUBICACION == "Aceptado" and element_layer_new.iloc[index_new_array].TIPO_TEMATIZACION == "ELIMINAR":
           # print("Se detectó un cambio en para el índice: " + str(index_new_array))
           # print("El elemento tiene los siguientes datos: ")
           # print(element_layer_new.iloc[index_new_array])
            element = element_layer_new.iloc[index_new_array].TIPO_ELEMENTO
            fecha = str(datetime.datetime.now())[0:16]
            point= element_layer_new.iloc[index_new_array].SHAPE
            punto = str(point.x)+","+str(point.y)+","+str(point.spatialReference["wkid"])
            send_mail(element,fecha,punto, "eliminación", "Eliminación")
    element_layer_old = element_layer_new
    time.sleep(1)    
    gis = GIS("https://gistest.esval.cl/portal", "siteadmin", "tvt.2021", verify_cert=False)
    reubicacion_item = gis.content.get("d2f19956d2e948bebaa67919cba50e54")
    reubicacion_layer = reubicacion_item.layers
    element_layer_new = reubicacion_layer[0].query(as_df=True)